# Connecting to a Postgres Database 

In [17]:
# Import Dependancies
from sqlalchemy import create_engine
import pandas as pd
import plotly.express as px
import numpy as np


### Create the database URI

In [18]:
DATABASE_URI = 'postgres+psycopg2://postgres:PW@localhost:5432/Employee'

### Create the Engine

In [19]:
engine = create_engine(DATABASE_URI)

### Connect to the Engine

In [20]:
connection = engine.connect()

### Run some raw queries

In [38]:
# Select 
r = connection.execute("""SELECT salary FROM Salaries""")

In [39]:
salaries_list = []
for row in r:
    salaries_list.append(row);
    # print(row)

df = pd.DataFrame(salaries_list, columns={"Salaries"});
#df


In [50]:
min_salary = df['Salaries'].min();
max_salary = df['Salaries'].max();

### Create a histogram to visualize the most common salary ranges for employees.

In [53]:
bin_size = 10000
# create the bins
counts, bins = np.histogram(df.Salaries, bins=range(min_salary-bin_size, max_salary+bin_size, bin_size))

bins = 0.5 * (bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'Salary Range', 'y':'Employees'})

fig.update_layout(
    title="The Most Common Salary Ranges for Employees",
    xaxis_title="Salary Range (USD, in bins of $10000)",
    yaxis_title="Number of Employees"
)


fig.show()
fig.write_image("../images/salaryRanges.png")


### Create a bar chart of average salary by title

In [54]:
r = connection.execute( """SELECT title, 
ROUND(AVG(salary),2) AS "ave_salary" 
FROM titles t, salaries s
WHERE t.emp_no = s.emp_no
GROUP BY title
ORDER BY "ave_salary" DESC;""")

In [55]:
ave_salaries_list = []
for row in r:
    ave_salaries_list.append(row);
    print(row)

df = pd.DataFrame(ave_salaries_list, columns={"Title", "Average Salary"});
df

('Senior Staff', Decimal('58503.29'))
('Staff', Decimal('58465.27'))
('Manager', Decimal('51531.04'))
('Technique Leader', Decimal('48580.51'))
('Engineer', Decimal('48539.78'))
('Senior Engineer', Decimal('48506.75'))
('Assistant Engineer', Decimal('48493.20'))


,Title,Average Salary
0,Senior Staff,58503.29
1,Staff,58465.27
2,Manager,51531.04
3,Technique Leader,48580.51
4,Engineer,48539.78
5,Senior Engineer,48506.75
6,Assistant Engineer,48493.20


In [57]:

fig = px.bar(df, x='Title', y='Average Salary',
             hover_data=['Title', 'Average Salary'], color='Average Salary',
             labels={'sal':'Average Salary By Title'}, height=400)

fig.update_layout(
    title="Average Salary By Title",
    xaxis_title="Employee Title",
)

fig.show()
fig.write_image("../images/SalaryByTitle.png")
